# Tutorial 4: Agents in LangChain

In this tutorial, we'll explore Agents in LangChain, which are autonomous entities capable of using tools and making decisions to accomplish tasks.

In [4]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.agents import load_tools, initialize_agent, AgentType,create_react_agent, AgentExecutor
from langchain.tools import Tool,DuckDuckGoSearchRun
# from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
# from langchain_community.tools.playwright.utils import create_sync_playwright_browser
# from langgraph.prebuilt.chat_agent_executor import create_react_agent

# Load environment variables
load_dotenv()

# Initialize Groq LLM
llm = ChatGroq(
        model_name="llama-3.1-70b-versatile",
        temperature=0.7,
        model_kwargs={"top_p": 0.8, "seed": 1337}
    )

## 1. Understanding Agent Architecture

Agents in LangChain combine an LLM with a set of tools to perform tasks. The LLM decides which tools to use and how to interpret their outputs.

## 2. Exploring Different Types of Agents

### Zero-shot React Agent

In [5]:
search = DuckDuckGoSearchRun()

def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

tools_web_search = [
    Tool(
    name="WordLength",
    func=get_word_length,
    description="Useful for getting the length of a word"
)
]

tools = load_tools(["wikipedia", "llm-math",Tool(
    name="WordLength",
    func=get_word_length,
    description="Useful for getting the length of a word"
)], llm=llm)

tools.extend(tools_web_search)

zero_shot_agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

zero_shot_agent.invoke(
    "What is the square root of the year Pythagoras was born?"
)

/tmp/ipykernel_79922/2121938572.py:22: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  zero_shot_agent = initialize_agent(




> Entering new AgentExecutor chain...
Thought: To find the square root of the year Pythagoras was born, I first need to find the year Pythagoras was born. 
Action: wikipedia
Action Input: Pythagoras birth year
Observation: Page: Pythagoras
Summary: Pythagoras of Samos (Ancient Greek: Πυθαγόρας; c. 570 – c. 495 BC) was an ancient Ionian Greek philosopher, polymath, and the eponymous founder of Pythagoreanism. His political and religious teachings were well known in Magna Graecia and influenced the philosophies of Plato, Aristotle, and, through them, the West in general. Knowledge of his life is clouded by legend; modern scholars disagree regarding Pythagoras's education and influences, but they do agree that, around 530 BC, he travelled to Croton in southern Italy, where he founded a school in which initiates were sworn to secrecy and lived a communal, ascetic lifestyle.
In antiquity, Pythagoras was credited with many mathematical and scientific discoveries, including the Pythagorean 

{'input': 'What is the square root of the year Pythagoras was born?',
 'output': 'The square root of the year Pythagoras was born is approximately 23.87.'}

### Conversational Agent

In [3]:
conversational_agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True
)

conversational_agent.invoke({"input":"Hello! I'd like to learn about the theory of relativity and then solve a math problem related to it.","chat_history":""})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: wikipedia
Action Input: theory of relativity
Observation: Page: Theory of relativity
Summary: The theory of relativity usually encompasses two interrelated physics theories by Albert Einstein: special relativity and general relativity, proposed and published in 1905 and 1915, respectively. Special relativity applies to all physical phenomena in the absence of gravity. General relativity explains the law of gravitation and its relation to the forces of nature. It applies to the cosmological and astrophysical realm, including astronomy.
The theory transformed theoretical physics and astronomy during the 20th century, superseding a 200-year-old theory of mechanics created primarily by Isaac Newton. It introduced concepts including 4-dimensional spacetime as a unified entity of space and time, relativity of simultaneity, kinematic and gravitational time dilation, and length contraction. In the field of ph

{'input': "Hello! I'd like to learn about the theory of relativity and then solve a math problem related to it.",
 'chat_history': '',
 'output': "I've learned about the theory of relativity from Wikipedia. Now, what math problem related to it would you like to solve?"}

### STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION Agent

In [8]:
plan_and_execute_agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

plan_and_execute_agent.run(
    "Find out who invented the telephone and calculate the number of years between that invention and the first moon landing."
)



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to find out who invented the telephone and when it was invented. I also need to find out when the first moon landing occurred.

Action:
```
{
  "action": "wikipedia",
  "action_input": {"query": "invention of the telephone"}
}
```

Observation: Page: Invention of the telephone
Summary: The invention of the telephone was the culmination of work done by more than one individual, and led to an array of lawsuits relating to the patent claims of several individuals and numerous companies. Notable people included in this were Antonio Meucci, Philipp Reis, Simon Alles, Elisha Gray and Alexander Graham Bell.



Page: History of the telephone
Summary: This history of the telephone chronicles the development of the electrical telephone, and includes a brief overview of its predecessors.  The first telephone patent was granted to Alexander Graham Bell in 1869.



Page: Timeline of the telephone
Summary: This timeline

'The telephone was invented by multiple individuals, but Alexander Graham Bell was granted the first telephone patent in 1869. The first moon landing occurred on July 20, 1969. Therefore, the number of years between the invention of the telephone and the first moon landing is 100 years.'

## 3. Creating Custom Tools for Agents

In [6]:
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

word_length_tool = Tool(
    name="WordLength",
    func=get_word_length,
    description="Useful for getting the length of a word"
)

custom_tools = [word_length_tool] + tools

custom_agent = initialize_agent(
    custom_tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

custom_agent.run(
    "What's the length of the word 'supercalifragilisticexpialidocious' and its square root?"
)

/tmp/ipykernel_79922/1727393380.py:20: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  custom_agent.run(




> Entering new AgentExecutor chain...
Thought: To answer this question, I need to find the length of the word 'supercalifragilisticexpialidocious' and its square root. First, I'll find the length of the word.

Action: WordLength
Action Input: supercalifragilisticexpialidocious
Observation: 34
Thought:Thought: Now that I have the length of the word, I need to find its square root. I'll use the Calculator to find the square root.

Action: Calculator
Action Input: sqrt(34)
Observation: Answer: 5.830951894845301
Thought:Question: What's the length of the word 'supercalifragilisticexpialidocious' and its square root?
Thought: To answer this question, I need to find the length of the word 'supercalifragilisticexpialidocious' and its square root. First, I'll find the length of the word.

Action: WordLength
Action Input: supercalifragilisticexpialidocious
Observation: 34
Thought:Question: What's the length of the word 'supercalifragilisticexpialidocious' and its square root?
Thought: To answ

KeyboardInterrupt: 

## 4. Implementing a Multi-tool Agent for Task Solving

In [8]:
# Combine all tools
all_tools = tools + tools_web_search + [word_length_tool]

multi_tool_agent = initialize_agent(
    all_tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

multi_tool_agent.run(
    """Please perform the following tasks:
    1. Find the current temperature in New York City.
    2. Calculate the square root of that temperature.
    3. Find a famous quote about weather and tell me how many characters it has.
    Provide a summary of your findings.
    """
)



> Entering new AgentExecutor chain...
Thought: To find the current temperature in New York City, I will use the DuckDuckGo Search tool.

Action:
```
{
  "action": "DuckDuckGo Search",
  "action_input": "current temperature in New York City"
}
```

Observation: Current conditions at New York City, Central Park (KNYC) Lat: 40.78°NLon: 73.97°WElev: 154ft. Fair. 59°F. 15°C. ... Local Forecast Office More Local Wx 3 Day History Hourly Weather Forecast. Extended Forecast for New York NY . Tonight. Low: 52 °F. Partly Cloudy. Tuesday. High: 65 °F. Partly Sunny ... Hourly Weather Forecast. National ... This is Winter Weather Awareness Week - October 27th through November 2nd ... New England and New York NWS Offices Will Continue an Experiment to Improve Frost-Freeze Services This Fall ; Current conditions at New York City, Central Park (KNYC) Lat: 40.78°NLon: 73.97°WElev: 154ft. Overcast. 62°F. 17°C. Humidity: 70%: A mix of clouds and sun in the morning followed by cloudy skies during the aft

"The current temperature in New York City is 62°F. The square root of this temperature is approximately 7.87. A famous quote about weather is 'There is no such thing as bad weather, only different kinds of good weather' by John Ruskin, which has 76 characters."

## Conclusion

In this tutorial, we've explored Agents in LangChain, including different types of agents, creating custom tools, and implementing a multi-tool agent for complex task solving. Agents provide a powerful way to create autonomous systems that can leverage various tools and make decisions to accomplish tasks.